In [1]:
from __future__ import print_function

In [2]:
import numpy as np

In [36]:
from keras.models import Model
from keras.layers import Input,Dense,BatchNormalization,Reshape, Convolution2D, Activation, Flatten, UpSampling2D,Dropout

In [43]:
gen_input_dim = 128
batch_size=100
drop_rate = 0.2
img_shape = [28,28,1]

In [17]:
def switch_training(model, is_trainable):
    model.trainable = is_trainable
    for layer in model.layers:
        layer.trainable = is_trainable

In [30]:

num_channel = 100
input_data = Input(shape=[gen_input_dim])
X = Dense(num_channel*14*14, activation='relu')(input_data)
X = Reshape([num_channel,14,14])(X)
X = UpSampling2D(size=(2,2))(X)
X = Convolution2D(int(num_channel/2), 3, 3, border_mode='same')(X)
X = BatchNormalization()(X)
X = Convolution2D(int(num_channel/4), 3, 3, border_mode='same')(X)
X = BatchNormalization()(X)
X = Convolution2D(1, 1, 1, border_mode='same')(X)
Y = Activation('sigmoid')(X)

generator = Model(inputs=[input_data], outputs=Y )
generator.compile(optimizer="adam", loss="binary_crossentropy")
    


C:\Users\Vishnu\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (3, 3), padding="same")`
C:\Users\Vishnu\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, (3, 3), padding="same")`
C:\Users\Vishnu\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (1, 1), padding="same")`


In [44]:

input_data = Input(shape=img_shape)
X = Convolution2D(256, 5, 5, subsample=(2, 2), border_mode = 'same', activation='relu')(input_data)
X = Dropout(drop_rate)(X)
X = Convolution2D(512, 5, 5, subsample=(2, 2), border_mode = 'same', activation='relu')(X)
X = Dropout(drop_rate)(X)
X = Flatten()(X)
X = Dense(256, activation='relu')(X)
X = Dropout(drop_rate)(X)
Y = Dense(2,activation='softmax')(X)

discriminator = Model(inputs=[input_data], outputs=Y)
discriminator.compile(loss='categorical_crossentropy', optimizer="adam")


    


C:\Users\Vishnu\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 5), strides=(2, 2), activation="relu", padding="same")`
  app.launch_new_instance()
C:\Users\Vishnu\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (5, 5), strides=(2, 2), activation="relu", padding="same")`


In [45]:


switch_training(discriminator, False)

input_data = Input(shape=[gen_input_dim])
X = generator(input_data)
Y = discriminator(X)
GAN = Model(inputs=input_data, outputs=Y)
GAN.compile(loss='categorical_crossentropy', optimizer="adam")
